# TP02 — Forward Kinematics of Robot Manipulators

**GEII3 — Industrie 4.0: Robotique | Lab Session 2**

**Duration**: 2 sessions × 4 hours  
**Prerequisites**: Homogeneous transformations, rotation matrices, Python/NumPy basics

---

## Learning Objectives

By the end of this lab (8 hours total), you should be able to:

1. Apply **three different parameterization methods** for robot forward kinematics:
   - Standard Denavit-Hartenberg (DH)
   - Modified Denavit-Hartenberg (MDH / Craig's convention)
   - Elementary Transform Sequence (ETS)

2. Understand the **advantages and limitations** of each approach

3. Implement FK algorithms **from scratch** in Python

4. Validate implementations against **Robotics Toolbox** (Peter Corke)

5. Analyze **workspace, singularities**, and compare robot architectures

6. Gain practical experience with industrial robots: **2R planar, 3R anthropomorphic, SCARA, PUMA 560**

---

## Lab Organization

### **Session 1** (4 hours)
- **Part I**: Theory recap & helper functions (30 min)
- **Exercise 1**: 2R planar robot - all three methods (60 min)
- **Exercise 2**: 3-DOF anthropomorphic robot (90 min)
- **Exercise 3**: SCARA robot (60 min)

### **Session 2** (4 hours)
- **Exercise 4**: PUMA 560 (6-DOF industrial robot) (120 min)
- **Exercise 5**: Comparative analysis & methodology discussion (60 min)
- **Exercise 6**: Workspace analysis & singularities (60 min)

---

## Required Libraries

```python
numpy, matplotlib, roboticstoolbox-python, spatialmath-python, scipy, sympy
```

In [ ]:
# Uncomment to install (takes ~2 minutes):
# !pip install -q numpy==1.26.4 roboticstoolbox-python spatialmath-python matplotlib scipy pandas sympy

---

# Part I: Theoretical Background

## Three Approaches to Forward Kinematics

### 1. Standard Denavit-Hartenberg (DH) Convention

The **standard DH convention** (introduced in 1955) uses 4 parameters per link:

| Parameter | Description |
|-----------|-------------|
| $\theta_i$ | Joint angle: rotation about $z_{i-1}$ axis |
| $d_i$ | Link offset: translation along $z_{i-1}$ axis |
| $a_i$ | Link length: translation along $x_i$ axis |
| $\alpha_i$ | Link twist: rotation about $x_i$ axis |

**Transformation sequence**: $R_z(\theta_i) \cdot T_z(d_i) \cdot T_x(a_i) \cdot R_x(\alpha_i)$

**Transformation matrix**:
$$
{}^{i-1}T_i = \begin{bmatrix}
\cos\theta_i & -\sin\theta_i\cos\alpha_i & \sin\theta_i\sin\alpha_i & a_i\cos\theta_i \\
\sin\theta_i & \cos\theta_i\cos\alpha_i & -\cos\theta_i\sin\alpha_i & a_i\sin\theta_i \\
0 & \sin\alpha_i & \cos\alpha_i & d_i \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

**Advantages**: Widely used, compact representation  
**Disadvantages**: Frame assignment can be tricky; issues with parallel axes

---

### 2. Modified Denavit-Hartenberg (MDH / Craig's Convention)

Same 4 parameters but **different frame attachment** (frames attached to joints, not links):

**Transformation sequence**: $R_x(\alpha_{i-1}) \cdot T_x(a_{i-1}) \cdot R_z(\theta_i) \cdot T_z(d_i)$

**Transformation matrix**:
$$
{}^{i-1}T_i = \begin{bmatrix}
\cos\theta_i & -\sin\theta_i & 0 & a_{i-1} \\
\sin\theta_i\cos\alpha_{i-1} & \cos\theta_i\cos\alpha_{i-1} & -\sin\alpha_{i-1} & -d_i\sin\alpha_{i-1} \\
\sin\theta_i\sin\alpha_{i-1} & \cos\theta_i\sin\alpha_{i-1} & \cos\alpha_{i-1} & d_i\cos\alpha_{i-1} \\
0 & 0 & 0 & 1
\end{bmatrix}
$$

**Advantages**: Frame $i$ attached to joint $i$ (more intuitive); used by RTB  
**Disadvantages**: Less common in textbooks; different from standard DH

---

### 3. The Elementary Transform Sequence (ETS)

The **Elementary Transform Sequence (ETS)** provides a universal method for describing the kinematics of any manipulator. An ETS comprises a sequence of elementary transforms $\mathbf{E}_i$ -- translations and rotations -- from the base frame to the robot's end-effector.

$$
\mathbf{E}_i = 
\left\{
\begin{matrix}
\mathbf{T}_{t_x}(\eta_i) \\
\mathbf{T}_{t_y}(\eta_i) \\
\mathbf{T}_{t_z}(\eta_i) \\
\mathbf{T}_{R_x}(\eta_i) \\
\mathbf{T}_{R_y}(\eta_i) \\
\mathbf{T}_{R_z}(\eta_i)
\end{matrix}
\right.
$$

Where the parameter $\eta_i$ is either:
- A **constant** $c_i$ (fixed translational offset or rotation)
- A **joint variable** $q_j(t)$ (revolute angle $\theta_j(t)$ or prismatic distance $d_j(t)$)

**Example**: For a 2-link planar robot:
$$
T = \mathbf{T}_{R_z}(q_1) \cdot \mathbf{T}_{t_x}(L_1) \cdot \mathbf{T}_{R_z}(q_2) \cdot \mathbf{T}_{t_x}(L_2)
$$

**Advantages**: 
- Extremely intuitive and easy to visualize
- Simple to implement and modify
- Direct mapping to physical robot structure
- No frame assignment ambiguity
- Supported by modern robotics libraries (Robotics Toolbox)

**Disadvantages**: 
- More transforms than DH (less compact)
- Newer approach (less common in older textbooks)

---

## Setup and Installation

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import cos, sin, pi, atan2, sqrt
from scipy.linalg import expm
from tp_02_fk_helpers import *  # type: ignore # Assuming helper functions are in this file

# Import Robotics Toolbox
try:
    import roboticstoolbox as rtb
    from spatialmath import SE3
    RTB_AVAILABLE = True
    print("✓ Robotics Toolbox loaded successfully")
    print(f"  RTB version: {rtb.__version__}")
except ImportError:
    RTB_AVAILABLE = False
    print("⚠ Robotics Toolbox not available. Run installation cell above.")

np.set_printoptions(precision=4, suppress=True)
print(f"✓ NumPy version: {np.__version__}")
print(f"✓ Python libraries ready!\n")

---

# Exercise 1: 2R Planar Robot (All Three Methods)

**Estimated time**: 60 minutes

## Problem Description

Consider a simple **2-DOF planar robot** (2R) with:
- Two revolute joints rotating about vertical (Z) axes
- Link lengths: $L_1 = 0.4$ m, $L_2 = 0.3$ m
- All motion in the XY plane

## Task 1.1: Standard DH Table (15 min)

**Instructions**:
1. Draw the robot schematically with coordinate frames following **standard DH convention**
2. Complete the DH parameter table
3. Write the transformation matrices ${}^0T_1$ and ${}^1T_2$
4. Derive ${}^0T_2$ symbolically

### Your derivation:

**Sketch** (insert image or describe frame placement):

_[Draw or describe here]_

**DH Table** (Standard):

| Joint i | $\theta_i$ | $d_i$ | $a_i$ | $\alpha_i$ |
|---------|-----------|-------|-------|------------|
| 1       | ?         | ?     | ?     | ?          |
| 2       | ?         | ?     | ?     | ?          |

**Matrix expressions**:

${}^0T_1 = ?$

${}^1T_2 = ?$

${}^0T_2 = {}^0T_1 \cdot {}^1T_2 = ?$

---

## Task 1.2: Implementation with Standard DH (10 min)

In [ ]:
# Robot parameters
L1 = 0.4  # m
L2 = 0.3  # m

# DH parameters for 2R robot (Standard convention)
dh_2r_standard = [
    # Format: (theta_offset, d_offset, a, alpha, joint_type)
    # TODO: Complete according to your table
]

# Test configuration: q1 = 30°, q2 = 45°
q_2r = np.array([np.deg2rad(30), np.deg2rad(45)])

# Compute FK with standard DH
T_2r_std, T_list_std = fk_dh(dh_2r_standard, q_2r, convention='standard')

print("\n" + "="*60)
print("  2R ROBOT - STANDARD DH")
print("="*60)
print(f"Configuration: q = [{np.rad2deg(q_2r[0]):.1f}°, {np.rad2deg(q_2r[1]):.1f}°]")
print(f"\nEnd-effector position:")
print(f"  x = {T_2r_std[0,3]:.4f} m")
print(f"  y = {T_2r_std[1,3]:.4f} m")
print(f"  z = {T_2r_std[2,3]:.4f} m")
print(f"\nFull transformation matrix T_0^2:")
print(T_2r_std)

# Visualize
fig, ax = plt.subplots(figsize=(8, 8))
plot_robot_2d(T_list_std, ax, 
              title=f"2R Robot (Standard DH)\nq = {np.rad2deg(q_2r)}°",
              draw_frames=True)
plt.show()

## Task 1.3: Modified DH Table and Implementation (15 min)

**Instructions**:
1. Establish coordinate frames following **modified DH convention** (frames attached to joints)
2. Complete the MDH parameter table
3. Implement and verify

### Your derivation:

**Modified DH Table**:

| Joint i | $\theta_i$ | $d_i$ | $a_{i-1}$ | $\alpha_{i-1}$ |
|---------|-----------|-------|-----------|----------------|
| 1       | ?         | ?     | ?         | ?              |
| 2       | ?         | ?     | ?         | ?              |

**Note**: Pay attention to the indexing! In MDH, $a_{i-1}$ and $\alpha_{i-1}$ refer to the **previous** link.

---

In [ ]:
# DH parameters for 2R robot (Modified convention)
dh_2r_modified = [
# Format: (theta_offset, d_offset, a, alpha, joint_type)
# TODO: Complete according to your MDH table
]

# Compute FK with modified DH
T_2r_mdh, T_list_mdh = fk_dh(dh_2r_modified, q_2r, convention='modified')

print("\n" + "="*60)
print("  2R ROBOT - MODIFIED DH")
print("="*60)
print(f"Configuration: q = [{np.rad2deg(q_2r[0]):.1f}°, {np.rad2deg(q_2r[1]):.1f}°]")
print(f"\nEnd-effector position:")
print(f"  x = {T_2r_mdh[0,3]:.4f} m")
print(f"  y = {T_2r_mdh[1,3]:.4f} m")
print(f"  z = {T_2r_mdh[2,3]:.4f} m")
print(f"\nFull transformation matrix T_0^2:")
print(T_2r_mdh)

# Compare with standard DH
print("\n--- Comparison: Standard vs Modified DH ---")
diff = np.max(np.abs(T_2r_std - T_2r_mdh))
print(f"Maximum difference: {diff:.2e}")
if diff < 1e-10:
    print("✓ Both methods give the same result!")
else:
    print("✗ Results differ - check your parameters")

## Task 1.4: Elementary Transform Sequence (20 min)

**Instructions**:
1. Identify the sequence of **elementary transforms** from base to end-effector
2. Build the ETS by listing each rotation and translation in order
3. Implement using the ETS formula

### Your derivation:

**Physical description**:
- Start at base (origin)
- Rotate by $q_1$ about Z-axis → **Joint 1**
- Translate by $L_1$ along X-axis → **Link 1**
- Rotate by $q_2$ about Z-axis → **Joint 2**
- Translate by $L_2$ along X-axis → **Link 2** (end-effector)

**Elementary Transform Sequence**:

$$
T = \mathbf{T}_{R_z}(q_1) \cdot \mathbf{T}_{t_x}(L_1) \cdot \mathbf{T}_{R_z}(q_2) \cdot \mathbf{T}_{t_x}(L_2)
$$

**Observation**: This is extremely intuitive! We simply follow the physical structure of the robot.

**ETS List representation**:
```python
[
    (rotz,   0,   True),   # Rz(q1) - joint variable
    (transx, L1,  False),  # Tx(L1) - constant
    (rotz,   1,   True),   # Rz(q2) - joint variable
    (transx, L2,  False),  # Tx(L2) - constant
]
```

---

In [ ]:
# Define Elementary Transform Sequence for 2R robot
# Sequence: Rz(q1) -> Tx(L1) -> Rz(q2) -> Tx(L2)

ets_2r = [
    # TODO: Complete the ETS according to your derivation
]

print("\nElementary Transform Sequence:")
for i, (func, param, is_joint) in enumerate(ets_2r, 1):
    func_name = func.__name__
    if is_joint:
        print(f"  E{i}: {func_name}(q{param+1})")
    else:
        print(f"  E{i}: {func_name}({param})")

# Compute FK using ETS
T_2r_ets, T_list_ets = fk_ets(ets_2r, q_2r)

print("\n" + "="*60)
print("  2R ROBOT - ELEMENTARY TRANSFORM SEQUENCE")
print("="*60)
print(f"Configuration: q = [{np.rad2deg(q_2r[0]):.1f}°, {np.rad2deg(q_2r[1]):.1f}°]")
print(f"\nEnd-effector position:")
print(f"  x = {T_2r_ets[0,3]:.4f} m")
print(f"  y = {T_2r_ets[1,3]:.4f} m")
print(f"  z = {T_2r_ets[2,3]:.4f} m")
print(f"\nFull transformation matrix:")
print(T_2r_ets)

# Compare all three methods
print("\n" + "="*60)
print("  COMPARISON: All Three Methods")
print("="*60)
diff_std_ets = np.max(np.abs(T_2r_std - T_2r_ets))
diff_mdh_ets = np.max(np.abs(T_2r_mdh - T_2r_ets))
print(f"Standard DH vs ETS: max diff = {diff_std_ets:.2e}")
print(f"Modified DH vs ETS: max diff = {diff_mdh_ets:.2e}")

if diff_std_ets < 1e-10 and diff_mdh_ets < 1e-10:
    print("\n✓ Excellent! All three methods give identical results.")
else:
    print("\n✗ Discrepancies detected - review your derivations.")

## Task 1.5: Validation with RTB (10 min)

In [ ]:
if RTB_AVAILABLE:
    from roboticstoolbox import DHRobot, RevoluteMDH
    
    # Create 2R robot using RTB (uses Modified DH by default)
    robot_2r = DHRobot([
        RevoluteMDH(a=0,  d=0, alpha=0),
        RevoluteMDH(a=L1, d=0, alpha=0)
    ], name="2R Planar")
    
    print(robot_2r)
    
    # Compute FK with RTB
    T_rtb = robot_2r.fkine(q_2r)
    
    print("\n" + "="*60)
    print("  RTB Forward Kinematics")
    print("="*60)
    print(T_rtb)
    
    # Compare with our implementations
    print("\n--- Validation ---")
    compare_with_rtb(T_2r_std, T_rtb, verbose=True)
    compare_with_rtb(T_2r_ets, T_rtb, verbose=True)
    
    # Visualize with RTB (optional)
    # robot_2r.plot(q_2r, backend='pyplot', block=False)
    
else:
    print("RTB not available - skipping validation")

## Task 1.6: Analysis Questions (10 min)

Answer the following questions based on your experience with this exercise:

### Q1: Which method was easiest to apply for this simple 2R robot? Why?

_Your answer:_


### Q2: What are the main differences between Standard and Modified DH in terms of frame placement?

_Your answer:_


### Q3: For the ETS method, what is the advantage of following the physical structure directly?

_Your answer:_


### Q4: If we add a third revolute joint at the end-effector, which method would require the least modification? Why?

_Your answer:_


---